### Ice Sales Analysis

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [18]:
# Import the DataFrame

url = 'https://raw.githubusercontent.com/DHE42/ice-sales-analysis/refs/heads/main/games.csv'
games_df = pd.read_csv(url)

# Print Head

print(games_df.head())



                       Name Platform  Year_of_Release         Genre  NA_sales  \
0                Wii Sports      Wii           2006.0        Sports     41.36   
1         Super Mario Bros.      NES           1985.0      Platform     29.08   
2            Mario Kart Wii      Wii           2008.0        Racing     15.68   
3         Wii Sports Resort      Wii           2009.0        Sports     15.61   
4  Pokemon Red/Pokemon Blue       GB           1996.0  Role-Playing     11.27   

   EU_sales  JP_sales  Other_sales  Critic_Score User_Score Rating  
0     28.96      3.77         8.45          76.0          8      E  
1      3.58      6.81         0.77           NaN        NaN    NaN  
2     12.76      3.79         3.29          82.0        8.3      E  
3     10.93      3.28         2.95          80.0          8      E  
4      8.89     10.22         1.00           NaN        NaN    NaN  


The Games DataFrame has 11 columns. These go over 

1) Name
2) Gaming Platform
3) Release Year
4) Genre
5) North America Sales
6) Europe Sales
7) Japan Sales
8) Other Sales
9) Critic Score
10) User Score
and
11) Rating.

Before performing any operations, let's see what the data type for each column is.

In [19]:
print("Data Types of Each Column")
print()
print(games_df.dtypes)
print()

Data Types of Each Column

Name                object
Platform            object
Year_of_Release    float64
Genre               object
NA_sales           float64
EU_sales           float64
JP_sales           float64
Other_sales        float64
Critic_Score       float64
User_Score          object
Rating              object
dtype: object



Name, Platform, Genre, User_Score, and Rating are all stored as an object. User_Score and Rating should all be stored as string, and all values should be lower case, as well as removing leading and trailing spaces. Rating values are all stored as an object. User_Score and Rating should all be stored as string, and all values should be lower case, as well as removing leading and trailing spaces. Year_of_Release should also be stored as int64 since there's no need for a decimal in a year.

In [20]:

# Step 1: Clean Cateogorical Columns

# Convert column names to lower case, which ensures consistency and makes it easier to reference columns in the DataFrame.
games_df.columns = games_df.columns.str.lower()
# Convert all string values to lower case, strip leading/trailing spaces, and replace spaces with underscores
string_columns = games_df.select_dtypes(include=['object']).columns
for col in string_columns:
    games_df[col] = games_df[col].str.lower().str.strip().str.replace(' ', '_')
# Remove leading and trailing periods, question marks, and exclamation marks
for col in string_columns:
    games_df[col] = games_df[col].str.strip('.!?-')
for col in string_columns:
    games_df[col] = games_df[col].str.replace('-', '')

# Step 2: Handle missing values

# Handling Categorical Columns
games_df['platform'] = games_df['platform'].fillna('unknown')
games_df['genre'] = games_df['genre'].fillna('unknown')
games_df['rating'] = games_df['rating'].fillna('unknown')
games_df['name'] = games_df['name'].fillna('unknown')

# Handling Numerical Columns
# Replace NaN values in the year_of_release column with the mean of the column
mean_year = games_df['year_of_release'].mean(skipna=True)
games_df['year_of_release'] = games_df['year_of_release'].fillna(mean_year)
# Replace NaN values in the na_sales column with the median of the column
median_na_sales = games_df['na_sales'].median(skipna=True)
games_df['na_sales'] = games_df['na_sales'].fillna(median_na_sales)
# Replace NaN values in eu_sales column with the median of the column
median_eu_sales = games_df['eu_sales'].median(skipna=True)
games_df['eu_sales'] = games_df['eu_sales'].fillna(median_eu_sales)
# Replace NaN values in jp_sales column with the median of the column 
median_jp_sales = games_df['jp_sales'].median(skipna=True)
games_df['jp_sales'] = games_df['jp_sales'].fillna(median_jp_sales)
# Replace NaN values in other_sales column with the median of the column
median_other_sales = games_df['other_sales'].median(skipna=True)
games_df['other_sales'] = games_df['other_sales'].fillna(median_other_sales)
# Convert user_score to float to handle NaN values for later conversion to int64, as int64 has trouble handling NaN values
games_df['user_score'] = pd.to_numeric(games_df['user_score'], errors='coerce')
# Replace NaN values in the user_score column with the mean of the column
mean_user_score = games_df['user_score'].mean(skipna=True)
games_df['user_score'] = games_df['user_score'].fillna(mean_user_score)
# Replace NaN values in the critic_score column with median of the column
median_critic_score = games_df['critic_score'].median(skipna=True)
games_df['critic_score'] = pd.to_numeric(games_df['critic_score'], errors='coerce')
games_df['critic_score'] = games_df['critic_score'].fillna(median_critic_score)

# Step 3: Data Type Conversion

# Convert categorical columns to string.
# These columns are currently stored as object dtype, which is a generic type in pandas.
# Converting them to string dtype ensures consistency and allows for string-specific operations.
games_df['name'] = games_df['name'].astype('string')
games_df['platform'] = games_df['platform'].astype('string')
games_df['genre'] = games_df['genre'].astype('string')
games_df['rating'] = games_df['rating'].astype('string')


# Step 4: Convert applicable numerical columns to int64, which is appropriate for mathematical operations and analysis.
# This is the most space efficient manner, since these columns do not require decimal precision.
# Convert user_score to int64
games_df['user_score'] = games_df['user_score'].astype('int64')
# Convert critic_score to int64
games_df['critic_score'] = games_df['critic_score'].astype('int64')
# Convert year_of_release to int64
games_df['year_of_release'] = games_df['year_of_release'].astype('int64')

print("Data types after conversion:")
print(games_df.dtypes)
print()
print("DF Head After Cleaning:")
print(games_df.head())
print()
print("Unique Values in Each Column:")
print(games_df['genre'].unique())
print(games_df['platform'].unique())
print(games_df['rating'].unique())
print()

Data types after conversion:
name               string[python]
platform           string[python]
year_of_release             int64
genre              string[python]
na_sales                  float64
eu_sales                  float64
jp_sales                  float64
other_sales               float64
critic_score                int64
user_score                  int64
rating             string[python]
dtype: object

DF Head After Cleaning:
                       name platform  year_of_release        genre  na_sales  \
0                wii_sports      wii             2006       sports     41.36   
1          super_mario_bros      nes             1985     platform     29.08   
2            mario_kart_wii      wii             2008       racing     15.68   
3         wii_sports_resort      wii             2009       sports     15.61   
4  pokemon_red/pokemon_blue       gb             1996  roleplaying     11.27   

   eu_sales  jp_sales  other_sales  critic_score  user_score   rating  
0    

The next step will be to take care of NaN values. 